# Datasets ETL Preparation Notebook

In this project, three different datasets are linked in order to apply the ML Model on all information combined and be able to train and predict well. As a result, the following notebook cells will get the ETL process done on both datasets making them ready for Analytics and ML Prediction Model.

## 1. Import libraries and load datasets

There are three files to load into the notebook:

- The first file is items.csv and it is from **Amazon Cell Phones Reviews** Dataset containing information about mobile phones prices and reviews by users on Amazon last updated in 2019.

- The second file is Amazon_Unlocked_Mobile.csv and it is from **Amazon Reviews: Unlocked Mobile Phones** Dataset containing more reviews from users on Amazon about mobile phones but last updated in 2016.

- The third file is a dataset from **GSMArena** of more than 10,000 mobile phones with detailed specifications about them

For this project, only items.csv and gsm.csv will be needed.


In [1]:
# import libraries and load  datasets
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from collections import defaultdict
import re
import numpy as np
import difflib

mobiles_with_reviews = pd.read_csv('reviews-ds/items.csv')
mobiles_with_reviews_2 = pd.read_csv('reviews-ds/Amazon_Unlocked_Mobile.csv')
mobiles_features = pd.read_csv('mobile-features-ds/gsm.csv')

C:\Users\YOGA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (66,67,71,76,78,82) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## 2. Gather necessary data for required model and understand its structure

In [2]:
# Take a look at reviews dataframe
mobiles_with_reviews.head()

,asin,brand,title,url,image,rating,reviewUrl,totalReviews,price,originalPrice
0,B0000SX2UC,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
1,B0009N5L7K,Motorola,Motorola I265 phone,https://www.amazon.com/Motorola-i265-I265-phon...,https://m.media-amazon.com/images/I/419WBAVDAR...,3.0,https://www.amazon.com/product-reviews/B0009N5L7K,7,49.95,0.0
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,2.7,https://www.amazon.com/product-reviews/B000SKTZ0S,22,99.99,0.0
3,B001AO4OUC,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,3.3,https://www.amazon.com/product-reviews/B001AO4OUC,21,0.00,0.0
4,B001DCJAJG,Motorola,Motorola V365 no contract cellular phone AT&T,https://www.amazon.com/Motorola-V365-contract-...,https://m.media-amazon.com/images/I/61LYNCVrrK...,3.1,https://www.amazon.com/product-reviews/B001DCJAJG,12,149.99,0.0


In [3]:
# Look at column names and their datatypes
mobiles_with_reviews.dtypes

asin              object
brand             object
title             object
url               object
image             object
rating           float64
reviewUrl         object
totalReviews       int64
price            float64
originalPrice    float64
dtype: object

In [4]:
# Look at column names and their datatypes
mobiles_with_reviews_2.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [5]:
# Look at column names and their datatypes
mobiles_with_reviews_2.dtypes

Product Name     object
Brand Name       object
Price           float64
Rating            int64
Reviews          object
Review Votes    float64
dtype: object

In [6]:
# Take a look at features dataframe
mobiles_features.head()

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,launch_status,body_dimensions,body_weight,...,main_camera_dual_or_triple,battery_music_play,selfie_camera_triple,main_camera_v1,selfie_camera,camera,main_camera,network,battery_talk_time,battery_stand.by
0,Benefon,Vega,GSM,GSM 900,No,No,1999,Discontinued,145 x 56 x 23 mm (5.71 x 2.20 x 0.91 in),190 g (6.70 oz),...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,4 - 10 h,3 - 6 days
1,Garmin-Asus,nuvifone M10,GSM / HSPA,GSM 900 / 1800 / 1900,NaN,NaN,"2010, January. Released 2010, March",Discontinued,-,-,...,NaN,NaN,NaN,NaN,V2,NaN,NaN,GSM 850 / 1800 / 1900 - US version,Up to 8 h,Up to 600 h (2G) / Up to 600 h (3G)
2,Gigabyte,GSmart G1305 Boston,GSM / HSPA,GSM 850 / 900 / 1800 / 1900,NaN,NaN,"2010, April. Released 2010, April",Discontinued,116 x 56.8 x 12.4 mm (4.57 x 2.24 x 0.49 in),118 g (4.16 oz),...,NaN,NaN,NaN,NaN,V2,NaN,NaN,NaN,Up to 7 h 10 min,Up to 410 h
3,Gigabyte,GSmart,GSM / HSPA,GSM 900 / 1800,NaN,NaN,Not officially announced yet,Cancelled,103 x 54 x 13.4 mm (4.06 x 2.13 x 0.53 in),-,...,NaN,NaN,NaN,NaN,V2,NaN,NaN,NaN,NaN,NaN
4,Google,Pixel 4 XL,GSM / CDMA / HSPA / EVDO / LTE,GSM 850 / 900 / 1800 / 1900,NaN,NaN,"2019, October 15","Available. Released 2019, October 22",160.4 x 75.1 x 8.2 mm (6.31 x 2.96 x 0.32 in),193 g (6.81 oz),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CDMA 800 / 1900,NaN,NaN


In [7]:
# Look at columns and their datatypes
mobiles_features.dtypes

oem                   object
model                 object
network_technology    object
network_2g_bands      object
network_gprs          object
                       ...  
camera                object
main_camera           object
network               object
battery_talk_time     object
battery_stand.by      object
Length: 86, dtype: object

## 3. Remove duplicates and unnecessary columns

In [8]:
# check for duplicates in the dataset
print(mobiles_with_reviews.shape)
mobiles_with_reviews = mobiles_with_reviews.drop_duplicates()
print(mobiles_with_reviews.shape)

(720, 10)
(720, 10)


In [9]:
# remove unneccessary columns in reviews df
mobiles_revs_1_df = mobiles_with_reviews[['brand','title','totalReviews','rating','price']]

# adjust titles of mobiles to ensure consistency and facilitate correct matching
mobiles_revs_1_df['title'] = mobiles_revs_1_df['title'].str.replace('+', ' Plus ')
mobiles_revs_1_df['title'] = mobiles_revs_1_df['title'].str.replace('.0', '')
mobiles_revs_1_df['title'] = mobiles_revs_1_df['title'].str.replace(' Smartphone', '')
mobiles_revs_1_df['title'] = mobiles_revs_1_df['title'].str.replace(' Smarthphone', '')
mobiles_revs_1_df['title'] = mobiles_revs_1_df['title'].str.replace('Cellphone - ', '')
mobiles_revs_1_df['title'] = mobiles_revs_1_df['title'].str.replace('Cellphone ', '')

# df ready to clean !
mobiles_revs_1_df.head() 

<ipython-input-9-1d5db9ebb427>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mobiles_revs_1_df['title'] = mobiles_revs_1_df['title'].str.replace('+', ' Plus ')
<ipython-input-9-1d5db9ebb427>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mobiles_revs_1_df['title'] = mobiles_revs_1_df['title'].str.replace('.0', '')
<ipython-input-9-1d5db9ebb427>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,brand,title,totalReviews,rating,price
0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,14,3.0,0.00
1,Motorola,Motorola I265 phone,7,3.0,49.95
2,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,22,2.7,99.99
3,Motorola,Motorola i335 Cell Phone Boost Mobile,21,3.3,0.00
4,Motorola,Motorola V365 no contract cellular phone AT&T,12,3.1,149.99


In [10]:
# Add totalReviews col with value 1
mobiles_with_reviews_2['totalReviews'] = 1

# Make new df with the required columns
mobiles_revs_2_df = mobiles_with_reviews_2.copy()
mobiles_revs_2_df = mobiles_revs_2_df[['Brand Name','Product Name','totalReviews','Rating','Price']]

# Make the df columns like the mobiles_revs_1_df columns
mobiles_revs_2_df.columns = ['brand','title','totalReviews','rating','price']
mobiles_revs_2_df.head()

,brand,title,totalReviews,rating,price
0,Samsung,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",1,5,199.99
1,Samsung,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",1,4,199.99
2,Samsung,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",1,5,199.99
3,Samsung,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",1,4,199.99
4,Samsung,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",1,4,199.99


In [11]:
# Solve some data entry problems
mobiles_revs_2_df['brand'] = mobiles_revs_2_df['brand'].str.replace('Samsybg','Samsung')
mobiles_revs_2_df['brand'] = mobiles_revs_2_df['brand'].str.replace('Amazon.com,','Apple')

# Get all possible brands from features df
ref_brands = list(set(mobiles_features['oem']))

cleaned_brands = []
for brand in mobiles_revs_2_df['brand']:
    added = False
    for ref_brand in ref_brands:
        # Check if one of the reference brands in the current brand column value
        if(str(ref_brand).lower() in str(brand).lower()):
            # Add it to the new column list
            cleaned_brands.append(ref_brand)
            added = True
            break
    if(added == False):
        # Find the closest reference brands to the given column value
        closest_brand_match = difflib.get_close_matches(str(brand), ref_brands)
        if(len(closest_brand_match) != 0):
            # Add the closest one to the column list
            cleaned_brands.append(closest_brand_match[0])
        else:
            # Add the original given value
            cleaned_brands.append(brand)
# Adjust the brand column with the created list
mobiles_revs_2_df['brand'] = cleaned_brands

In [12]:
# Remove the brands that do not exist in the reference brands
mobiles_revs_2_df = mobiles_revs_2_df[mobiles_revs_2_df['brand'].isin(ref_brands)].reset_index(drop=True)

# Define the aggregation for the totalReviews, ratings and prices based on new column
f = {'totalReviews': ['sum'], 'rating': ['mean'], 'price': ['mean']}

# Use the brand and mob_name columns to aggregate and re-convert it to df
mobiles_revs_2_new_df = mobiles_revs_2_df.groupby(['brand','title']).agg(f).reset_index()
mobiles_revs_2_new_df.columns = ['brand','title','totalReviews','rating','price']
print(mobiles_revs_2_new_df.shape)

(3129, 5)


In [13]:
# Merge the 2 reviews dfs in one df
mobiles_revs_df = mobiles_revs_1_df.append(mobiles_revs_2_new_df)
mobiles_revs_df.shape

(3849, 5)

In [14]:
# Check for duplicates in features df
print(mobiles_features.shape)
mobiles_features = mobiles_features.drop_duplicates()
print(mobiles_features.shape)

# remove the + in mobile models and add " Plus" instead for consistency in df
mobiles_features['model'] = mobiles_features['model'].str.replace('+', ' Plus')
mobiles_features['model'] = mobiles_features['model'].replace(to_replace ='\((.*?)\)+', value = '', regex = True)
mobiles_features['model'] = mobiles_features['model'].str.replace(' CDMA', '')
mobiles_features['model'] = mobiles_features['model'].str.replace('.0', '')
mobiles_features['model'] = mobiles_features['model'].str.rstrip()

(10679, 86)
(10105, 86)


## 4. Merge Datasets

In order to merge these two datasets, the title column in the items.csv file needs to be cleaned and refined to identify the mobile model that was reviewed and easily link it to the gsm.csv model column

**To achieve that, the below steps were followed:**

- The models in gsm.csv are used as a key to search in title strings in order to find the mobile model
- The models found were saved in titles dict and the longest match is considered the most accurate
- The mobile name is added to the reviews df and used to re-aggregate the reviews information
- The two dfs are finally linked using the new column (mob_name)

In [15]:
# Get mobiles brands and names in a separate df to use it to clean the reviews df
gsmA_mobile_names = mobiles_features[['oem','model']]
gsmA_mobile_names.head(10)

,oem,model
0,Benefon,Vega
1,Garmin-Asus,nuvifone M
2,Gigabyte,GSmart G15 Boston
3,Gigabyte,GSmart
4,Google,Pixel 4 XL
5,Google,Pixel 4
6,Google,Pixel 3a XL
7,Google,Pixel 3a
8,Haier,N
9,Haier,T


In [16]:
# drop rows that has a missing title (mobile name) since we will not be able to reference this phone
mobiles_revs_df.dropna(subset = ['brand','title'], inplace=True)
mobiles_revs_df.shape

(3845, 5)

In [17]:
def create_closest_strings_dict(mobiles_revs_df,org_mobile_names):
    
    '''
    INPUTs
    mobiles_revs_df - reviews pandas dataframe 
    org_mobile_names - df with mobile brand (oem) and model
    
    OUTPUT
    titles_dict - a dict created with titles in reviews df as keys and their values 
                  are in a form of list that has all the matched models from the other df
    '''
    # create a dict that has lists as their values
    titles_dict = defaultdict(list)
    # Add all titles to a list to loop on
    mobile_names = list(mobiles_revs_df['title'])
    
    for title in mobile_names:
        
        # Get brand of the current title from df
        mobile_brand = mobiles_revs_df[mobiles_revs_df['title'] == title]['brand'].iloc[0]
        # Filter the org_mobile_names df on the brand of the title for model optimized searching
        possible_sim_names = list(org_mobile_names[org_mobile_names['oem'].str.lower() == mobile_brand.lower()]['model'])
        # make an initial empty list for title
        titles_dict[title] = []
        
        for model in possible_sim_names:
            # Check if the model in org_mobile_names matches inside the current title
            if (model.lower() in title.lower().replace('(','').replace(')','')):
                # Check if the model was not added before to the title dict
                if (model not in titles_dict[title]):
                    # Append the model to the title list
                    titles_dict[title].append(model)
                    
    return titles_dict # return the titles dict of all possible matches

In [18]:
# Get the dict of models matches from the gsm df in the reviews df
titles_dict = create_closest_strings_dict(mobiles_revs_df, gsmA_mobile_names)
print('Items with no matches:')
# Get titles that have no matches from gsm.csv
for key, values in titles_dict.items():
    if values == []:
        print(key,values)
titles_dict

Items with no matches:
Apple iPod Touch 16GB (5th Generation) - Space Gray (Renewed) []
AT&T Wireless 3G / 4G / LTE Micro SIM Card - Postpaid /Go Phone Prepaid - SKU 4681B Softcard SIM []
Nokia Lumia Unlock Code At&t All Models 520 635 830 920 925 1020 1520 Fast []
Amazon Fire 32GB AT&T Smart Phone / No Contract Ready To Activate On Your AT&T Service []
Apple iPad Pro ML0N2CL/A (128GB Wi-Fi Space Gray) (International Model no Warranty) []
ASUS ZenFone 2 Deluxe Special Edition, Unlocked Cellphone, 4GB RAM, 128GB (U.S. Warranty) (Discontinued by Manufacturer) []
ASUS ZenFone 2 Laser Unlocked Smartphone, 3GB RAM, 16GB Storage []
ASUS ZenFone 2 Laser Unlocked Smartphone, 3GB RAM, 32GB Storage []
ASUS ZenFone 2 Laser ZE500KL 16GB 5-inch Dual-SIM LTE FACTORY UNLOCKED International Stock No Warranty (GOLD) []
ASUS ZenFone 2 Laser ZE550KL 2GB/16GB 5.5-inch 4G LTE Dual SIM Factory Unlocked International Stock No Warranty (Red) []
ASUS ZenFone 2 Unlocked Cellphone, 16GB, Black []
ASUS ZenFone 2 

defaultdict(list,
            {'Motorola I265 phone': ['E', 'P', 'One', 'A'],
             'MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE CELL PHONE': ['E',
              'C168',
              'P',
              'One',
              'A'],
             'Motorola i335 Cell Phone Boost Mobile': ['E', 'P', 'One', 'A'],
             'Motorola V365 no contract cellular phone AT&T': ['E',
              'V',
              'P',
              'One',
              'A',
              'V3',
              'V36'],
             'Nokia 16 Black Phone (T-Mobile)': ['1', '6', '', '16', 'E', 'N'],
             'Samsung Alias2 U7 Phone, Black (Verizon Wireless)': ['Z',
              'E',
              'C',
              'S'],
             'Samsung a167 Prepaid GoPhone (AT&T)': ['E', 'S', 'A167', 'A1'],
             'Verizon Wireless Motorola RAZR V3m - Silver': ['E',
              'V',
              'A',
              'V3',
              'RAZR V3'],
             'Motorola Barrage V8 Phone (Verizon Wireless)

In [19]:
def get_nearest_matching_names(mobiles_dict):
    
    '''
    INPUTs
    mobiles_dict - titles dict with list of all possible matching models
    
    OUTPUT
    titles_longest_dict - a dict created with titles in reviews df as keys and their values 
                          are the longest model string matching the title key
    '''
    
    # create a new empty dict
    titles_longest_dict = dict()
    
    for title in mobiles_dict:
        # Check if there where matches found
        if(len(mobiles_dict[title]) != 0):
            # Get the longest string in the list
            longest_title = max(mobiles_dict[title], key=len)
            # Handle some special cases
            if(len(longest_title) > 2 and longest_title not in ('One','Galaxy J','Redmi','P Plus','Galaxy Note')):
                # Add the longest string to the new dict
                titles_longest_dict[title] = longest_title
    
    return titles_longest_dict # return the titles dict with their most matching model

In [20]:
# Get the closest matching model-title dict 
closest_matches_dict = get_nearest_matching_names(titles_dict)
closest_matches_dict

{'MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE CELL PHONE': 'C168',
 'Samsung a167 Prepaid GoPhone (AT&T)': 'A167',
 'Verizon Wireless Motorola RAZR V3m - Silver': 'RAZR V3',
 'Samsung T1G Prepaid Phone (Net)': 'T1G',
 'Samsung Focus I917 Unlocked Phone with Windows 7 OS, 5 MP Camera, and Wi-Fi--No Warranty (Black)': 'Focus',
 'Samsung Focus Flash I677 8GB Unlocked GSM Phone with Windows 7.5 OS, 5MP Camera, GPS, Wi-Fi, Bluetooth and FM Radio - Black': 'Focus Flash I677',
 'Nokia Lumia 0 Black Factory Unlocked': 'Lumia 0',
 'Motorola Droid RAZR M XT7 Verizon Wireless, 8GB, White': 'DROID RAZR M',
 'Nokia Lumia 9, Black 32GB (AT&T)': 'Lumia 9',
 'Nokia Lumia 9, Yellow 32GB (AT&T)': 'Lumia 9',
 'Nokia Lumia 822 16GB Windows Phone Black - Verizon': 'Lumia 822',
 'Nokia Lumia 822, White 16GB (Verizon Wireless)': 'Lumia 822',
 'Nokia Lumia 9 32GB Unlocked GSM Windows 8 w/ Carl Zeiss Optics Camera - Yellow': 'Lumia 9',
 'Samsung Galaxy S4 GT-I90 Factory Unlocked Cellphone, 16GB, White': 'Gala

In [21]:
def add_new_mobile_name(mobiles_revs_df, closest_matches_dict):
    
    '''
    INPUT
    mobiles_revs_df - mobile reviews df
    closest_matches_dict - titles dict with best matching model for each title
    
    OUTPUT
    new_mobile_rev_df - a copy of mobile reviews df but adding mob_name column which is the matching
                        model to each title in the row and none if no match found
    '''
    
    # Create a copy from mobile reviews df
    new_mobile_rev_df = mobiles_revs_df.copy()
    # list to append values
    names = [] 
    
    for title in mobiles_revs_df['title']: 
        # Check if a match was found for the current title
        if(title in closest_matches_dict):
            # Append matching value to names list
            names.append(closest_matches_dict[title])
        else:
            # Append None since no match found
            names.append('None')
    # Use the names list to create the new column
    new_mobile_rev_df['mob_name'] = names    
    # Remove the title column since it became useless
    del new_mobile_rev_df['title']
    
    return new_mobile_rev_df # return the new df with the mob_name column instead of title

In [28]:
# Create the new df with mob_name colimn
mobile_names_with_revs = add_new_mobile_name(mobiles_revs_df, closest_matches_dict)
# Remove the rows that had no match (mob_name = None)
mobile_names_with_revs = mobile_names_with_revs[mobile_names_with_revs['mob_name'] != 'None'].reset_index(drop=True)

# Initialize the weighted mean function to accurately aggregate the new matches 
# and re-calculate the ratings & prices
wm = lambda x: np.average(x, weights=mobile_names_with_revs.loc[x.index, "totalReviews"])
# Define the aggregation for the totalReviews, ratings and prices based on new column
f = {'totalReviews': 'sum', 'rating': wm, 'price': 'max'}
# Use the brand and mob_name columns to aggregate and re-convert it to df
mobile_names_with_revs_agg = mobile_names_with_revs.groupby(['brand','mob_name']).agg(f).reset_index()
mobile_names_with_revs_agg.head(10)

,brand,mob_name,totalReviews,rating,price
0,ASUS,ROG Phone,6,3.700000,629.00
1,ASUS,ZenFone Live,1,5.000000,94.45
2,ASUS,Zenfone 5z ZS6KL,97,4.300000,499.00
3,Acer,Liquid Jade Z,4,1.500000,129.99
4,Acer,Liquid M2,15,3.200000,34.95
5,Acer,Liquid Z2,1,4.000000,47.99
6,Acer,Liquid Z4,2,5.000000,114.11
7,Apple,Watch 38mm,81,4.395062,367.63
8,Apple,Watch 42mm,78,4.423077,775.00
9,Apple,Watch Sport 38mm,535,4.272897,269.99


In [29]:
# Remove duplicates from features df
mobiles_features_refined = mobiles_features.drop_duplicates()
# Remove duplicated mobile names to avoid replication during join
mobiles_features_refined = mobiles_features_refined.groupby(['oem','model']).first().reset_index()

# link features to cleaned mobile_reviews df
mobiles_info_merged = mobile_names_with_revs_agg.merge(mobiles_features_refined, how='inner',left_on=['brand','mob_name']
                                                       , right_on=['oem','model'])

# Adjust columns names
mobiles_info_merged.columns.values[0] = 'brand'
mobiles_info_merged.columns.values[1] = 'mob_name'
mobiles_info_merged.columns.values[2] = 'total_reviews'
mobiles_info_merged.columns.values[3] = 'rating'
mobiles_info_merged.columns.values[4] = 'price'

# Remove oem and model since they are replicas of brand and mob_name
del mobiles_info_merged['oem']
del mobiles_info_merged['model']

mobiles_info_merged.head()

,brand,mob_name,total_reviews,rating,price,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,...,main_camera_dual_or_triple,battery_music_play,selfie_camera_triple,main_camera_v1,selfie_camera,camera,main_camera,network,battery_talk_time,battery_stand.by
0,Acer,Liquid Jade Z,4,1.500000,129.99,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,"2015, March",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HSDPA 850 / 900 / 1900 / 2100,Up to 7 h (2G) / Up to 4 h (3G),Up to 410 h (2G) / Up to 400 h (3G)
1,Acer,Liquid M2,15,3.200000,34.95,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, March",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HSDPA 850 / 1700(AWS),Up to 4 h,Up to 200 h
2,Acer,Liquid Z2,1,4.000000,47.99,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, March",...,NaN,NaN,NaN,NaN,No,NaN,NaN,HSDPA 850 / 1700(AWS),Up to 4 h,Up to 450 h
3,Acer,Liquid Z4,2,5.000000,114.11,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, January",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LTE band 2(1900), 4(1700/2100), 7(2600), 17(700)",NaN,NaN
4,Apple,Watch 38mm,81,4.395062,367.63,No cellular connectivity,NaN,No,No,"2014, September",...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,Up to 18 h (mixed usage)


In [30]:
mobiles_info_merged.shape

(736, 89)

## 5. Refine Features Columns

After finishing the refinements in reviews df and merging the 2 datasets, it's now the turn for the features columns

The below steps are highly important:
- Remove useless features (with a lot of missing values or not necessary for mobile evaluation)
- Adjust useful features columns values to be able to be used in model

In [31]:
# Check for missing values in the features added columns
percent_missing = mobiles_info_merged.isnull().sum() * 100 / len(mobiles_info_merged)
percent_missing

brand                  0.000000
mob_name               0.000000
total_reviews          0.000000
rating                 0.000000
price                  0.815217
                        ...    
camera                97.418478
main_camera          100.000000
network               47.826087
battery_talk_time     22.282609
battery_stand.by      28.125000
Length: 89, dtype: float64

In [32]:
# Remove columns having more than 70% missing values since they would not be indicative or useful
mobiles_info_merged = mobiles_info_merged.loc[:, mobiles_info_merged.isnull().mean() < .30]
# Check remaining columns
percent_missing_2 = mobiles_info_merged.isnull().sum() * 100 / len(mobiles_info_merged)
percent_missing_2

brand                    0.000000
mob_name                 0.000000
total_reviews            0.000000
rating                   0.000000
price                    0.815217
network_technology       0.000000
network_2g_bands         1.086957
network_gprs             4.211957
network_edge             4.211957
launch_announced         0.000000
launch_status            0.000000
body_dimensions          0.000000
body_weight              0.000000
body_sim                 0.000000
display_type             0.000000
display_size             1.086957
display_resolution       0.407609
memory_card_slot         0.000000
sound_loudspeaker        0.000000
sound_3.5mm_jack         0.135870
comms_wlan               0.000000
comms_bluetooth          0.000000
comms_gps                0.000000
comms_radio              1.494565
comms_usb                0.679348
features_sensors         0.000000
misc_colors              0.000000
network_3g_bands        11.684783
network_speed           11.277174
platform_os   

In [33]:
# Display current df after merging and cleaning
mobiles_info_merged

,brand,mob_name,total_reviews,rating,price,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,...,platform_gpu,memory_internal,main_camera_single,main_camera_video,misc_price,main_camera_features,selfie_camera_single,battery,battery_talk_time,battery_stand.by
0,Acer,Liquid Jade Z,4,1.500000,129.99,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,"2015, March",...,Mali-T760MP2,"8 GB, 1 GB RAM - Liquid Jade Z16 GB, 2 GB RAM ...","13 MP, AF, f/1.8",1080p@30fps,About 200 EUR,"LED flash, HDR, panorama",5 MP,Non-removable Li-Po 2300 mAh battery,Up to 7 h (2G) / Up to 4 h (3G),Up to 410 h (2G) / Up to 400 h (3G)
1,Acer,Liquid M2,15,3.200000,34.95,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, March",...,Adreno 302,"4 GB, 512 MB RAM - Liquid M2208 GB, 1 GB RAM -...","5 MP, AF",Yes,About 80 EUR,"LED flash, panorama",2 MP,Removable Li-Ion 1300 mAh battery,Up to 4 h,Up to 200 h
2,Acer,Liquid Z2,1,4.000000,47.99,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, March",...,Adreno 302,"8 GB, 1 GB RAM","5 MP, AF",720p@30fps,About 90 EUR,LED flash,2 MP,Removable Li-Ion 1300 mAh battery,Up to 4 h,Up to 450 h
3,Acer,Liquid Z4,2,5.000000,114.11,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, January",...,Mali-T760MP2,"8 GB, 1 GB RAM16 GB, 2 GB RAM","5 MP, AF","Yes, dual shot",About 130 EUR,LED flash,2 MP,Removable Li-Po 2000 mAh battery,NaN,NaN
4,Apple,Watch 38mm,81,4.395062,367.63,No cellular connectivity,NaN,No,No,"2014, September",...,PowerVR SGX543,"8 GB, 512 MB RAM",NaN,NaN,About 650 EUR,NaN,NaN,Non-removable Li-Ion 205 mAh battery (0.78 Wh),NaN,Up to 18 h (mixed usage)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,alcatel,Pop 3,17,3.352941,77.10,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2016, February",...,NaN,"8 GB, 1 GB RAM","5 MP or 8 MP or 13 MP, AF",1080p@30fps,About 120 EUR,"LED flash, HDR, panorama",2 MP or 5 MP or 8 MP,Non-removable Li-Ion 2500/2910 mAh battery,NaN,NaN
732,alcatel,Pop 4 Plus,1,4.000000,79.99,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2016, February",...,Adreno 304,"16 GB, 1.5 GB RAM","8 MP, AF, f/2.0",720p@30fps,About 110 EUR,"LED flash, HDR, panorama",5 MP,Removable Li-Ion 2500 mAh battery,Up to 18 h (2G) / Up to 15 h (3G),Up to 525 h (2G) / Up to 500 h (3G)
733,alcatel,Pop C1,1,3.000000,32.99,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2013, September",...,NaN,"4 GB, 512 MB RAM",2 MP,Yes,About 70 EUR,NaN,NaN,Removable Li-Ion 1300 mAh battery,Up to 6 h 48 min (2G) / Up to 2 h 48 min (3G),Up to 433 h (2G) / Up to 325 h (3G)
734,verykool,i121,5,2.600000,10.07,GSM,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,No,2013. Released 2013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Removable Li-Ion 650 mAh battery,NaN,NaN


After taking a look at all the available features, I decided to choose the below columns to process and use as the most indicative and useful features to access the mobile phone:

- price
- launch_announced
- network_technology
- body_dimensions
- body_weight
- body_sim
- display_size
- platform_os
- memory_internal
- selfie_camera_single
- battery

This will require a different way to refine based on their values

In [34]:
# Remove mobiles with less than 3 reviews since it would not be indicative
mobiles_info_merged = mobiles_info_merged[mobiles_info_merged['total_reviews'] > 2].reset_index(drop=True)
mobiles_info_merged.shape

(668, 42)

In [35]:
# Define list of the required cols from the mobiles_info_merged df to facilitate selecting at the end
required_cols = ['brand','mob_name']

In [36]:
# Check for price type and values
print(mobiles_info_merged['price'].dtype)
print(mobiles_info_merged['price'].value_counts())

# Convert zeroes to NANs to avoid it to be misleading to the model
mobiles_info_merged['price'] = mobiles_info_merged['price'].replace({0:np.nan})
if('price' not in required_cols):
    required_cols.append('price')

float64
2000.00    14
0.00        8
129.99      6
99.99       6
59.99       5
           ..
351.09      1
84.99       1
135.99      1
209.99      1
269.95      1
Name: price, Length: 522, dtype: int64


In [37]:
# Check for launch_announced type and values
print(mobiles_info_merged['launch_announced'].dtype)
print(mobiles_info_merged['launch_announced'].value_counts())

object
2014, September                            20
2015, September                            18
2015, March                                14
2019, February                             13
2016, February                             13
                                           ..
2020, April 27                              1
2011, June. Released 2011, July             1
2012, June. Released 2012, September        1
2009, July. Released 2009, July             1
2013, September. Released 2013, October     1
Name: launch_announced, Length: 284, dtype: int64


In [38]:
# Get launch year and add it to new column launch_year
# Split on comma and expand
launch_date = mobiles_info_merged['launch_announced'].str.split(',',expand=True)
# Get the year value
mobiles_info_merged['launch_year'] = launch_date[0]
# Change the values to numeric removing any non-numeric value
mobiles_info_merged['launch_year'] = pd.to_numeric(mobiles_info_merged['launch_year'], errors='coerce')
# Check for results
print(mobiles_info_merged['launch_year'].dtype)
print(mobiles_info_merged['launch_year'].value_counts())

# Add the col to required_cols
if('launch_year' not in required_cols):
    required_cols.append('launch_year')

float64
2015.0    101
2014.0     96
2013.0     62
2016.0     56
2011.0     53
2012.0     51
2019.0     46
2018.0     40
2010.0     39
2017.0     31
2009.0     27
2008.0     19
2007.0     17
2006.0      8
2020.0      6
2005.0      6
2004.0      2
2003.0      1
1998.0      1
Name: launch_year, dtype: int64


In [39]:
# Check for network_technology type and values
print(mobiles_info_merged['network_technology'].dtype)
print(mobiles_info_merged['network_technology'].value_counts())

object
GSM / HSPA / LTE                  238
GSM / HSPA                        221
GSM / CDMA / HSPA / EVDO / LTE     63
GSM                                62
GSM / CDMA / HSPA / LTE            26
CDMA / EVDO                        24
GSM / CDMA / HSPA / EVDO            9
No cellular connectivity            8
CDMA / EVDO / LTE                   5
CDMA / CDMA2000                     3
GSM / UMTS                          3
GSM / HSPA / EVDO / LTE             2
CDMA                                2
CDMA / HSPA / EVDO / LTE            1
GSM / UMTS / HSPA                   1
Name: network_technology, dtype: int64


In [40]:
# Get technologies of each mobile from df column
technologies = mobiles_info_merged['network_technology'].str.split(' / ')
# Get all posible technologies as a reference
all_choices = max(technologies, key=len)
all_choices

['GSM', 'CDMA', 'HSPA', 'EVDO', 'LTE']

In [41]:
def create_categorial_vars_from_networks(technologies,all_choices):
    
    '''
    INPUT
    technologies - list of lists of technologies for all mobiles in df
    all_choices - list of all posible technologies as a reference
    
    OUTPUT
    new_technologies_list - a list of lists of 0s and 1s indicating if the mobile is supporting the technologies or not
    '''
    new_technologies_list = []
    for tech_list in technologies:
        t_list_nums = []
        for technology in all_choices:
            if(technology in tech_list):
                t_list_nums.append(1)
            else:
                t_list_nums.append(0)
        new_technologies_list.append(t_list_nums)
    return new_technologies_list

In [42]:
# Get the col values as list of lists (0s and 1s) indicating the technologies supported by the mobiles
new_technologies_list = create_categorial_vars_from_networks(technologies,all_choices)
# Change the list into dataframe with the names of the technologies
techs_df = pd.DataFrame(new_technologies_list,columns =all_choices)
# Add the new cols to the original df
mobiles_info_with_techs = pd.concat([mobiles_info_merged, techs_df], axis=1)

# Add cols to required cols
if(all_choices[0] not in required_cols):
    required_cols.extend(all_choices)
# Show output    
mobiles_info_with_techs.head(10)

,brand,mob_name,total_reviews,rating,price,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,...,selfie_camera_single,battery,battery_talk_time,battery_stand.by,launch_year,GSM,CDMA,HSPA,EVDO,LTE
0,Acer,Liquid Jade Z,4,1.500000,129.99,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,"2015, March",...,5 MP,Non-removable Li-Po 2300 mAh battery,Up to 7 h (2G) / Up to 4 h (3G),Up to 410 h (2G) / Up to 400 h (3G),2015.0,1,0,1,0,1
1,Acer,Liquid M2,15,3.200000,34.95,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, March",...,2 MP,Removable Li-Ion 1300 mAh battery,Up to 4 h,Up to 200 h,2015.0,1,0,1,0,0
2,Apple,Watch 38mm,81,4.395062,367.63,No cellular connectivity,NaN,No,No,"2014, September",...,NaN,Non-removable Li-Ion 205 mAh battery (0.78 Wh),NaN,Up to 18 h (mixed usage),2014.0,0,0,0,0,0
3,Apple,Watch 42mm,78,4.423077,775.00,No cellular connectivity,NaN,No,No,"2014, September",...,NaN,Non-removable Li-Ion 250 mAh battery (0.94 Wh),NaN,Up to 18 h (mixed usage),2014.0,0,0,0,0,0
4,Apple,Watch Sport 38mm,535,4.272897,269.99,No cellular connectivity,NaN,No,No,"2014, September",...,NaN,Non-removable Li-Ion 205 mAh battery (0.78 Wh),NaN,Up to 18 h (mixed usage),2014.0,0,0,0,0,0
5,Apple,Watch Sport 42mm,33,4.666667,249.99,No cellular connectivity,NaN,No,No,"2014, September",...,NaN,Non-removable Li-Ion 250 mAh battery (0.94 Wh),NaN,Up to 18 h (mixed usage),2014.0,0,0,0,0,0
6,Apple,iPad 2,5,3.200000,129.60,CDMA / EVDO,CDMA 800 / 1900,No,No,"2011, March. Released 2011, March",...,VGA,Non-removable Li-Po 6930 mAh battery (25 Wh),NaN,NaN,2011.0,0,1,0,1,0
7,Apple,iPad Air,16,3.950000,379.99,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,Yes,Yes,"2019, March",...,"7 MP, f/2.2, 31mm (standard)",Non-removable Li-Po battery (30.2 Wh),Up to 10 h (multimedia),NaN,2019.0,1,0,1,0,1
8,Apple,iPad Air 2,124,4.200000,211.39,GSM / CDMA / HSPA / EVDO / LTE,GSM 850 / 900 / 1800 / 1900,Yes,Yes,"2014, October",...,"1.2 MP, f/2.2, 31mm (standard)",Non-removable Li-Po 7340 mAh battery (27.62 Wh),Up to 10 h (multimedia),NaN,2014.0,1,1,1,1,1
9,Apple,iPad mini 4,10,5.000000,249.99,GSM / CDMA / HSPA / EVDO / LTE,GSM 850 / 900 / 1800 / 1900,Yes,Yes,"2015, September",...,"1.2 MP, f/2.2, 31mm (standard)",Non-removable Li-Ion 5124 mAh battery (19.1 Wh),Up to 10 h (multimedia),NaN,2015.0,1,1,1,1,1


In [43]:
# Check for price type and values
print(mobiles_info_merged['body_dimensions'].dtype)
print(mobiles_info_merged['body_dimensions'].value_counts())

object
146 x 72 x 7.3 mm (5.75 x 2.83 x 0.29 in)         3
-                                                 3
126.1 x 65.4 x 11.3 mm (4.96 x 2.57 x 0.44 in)    3
156.6 x 78.7 x 7.9 mm (6.17 x 3.10 x 0.31 in)     2
42 x 35.9 x 10.5 mm (1.65 x 1.41 x 0.41 in)       2
                                                 ..
106.4 x 53.1 x 17.5 mm (4.19 x 2.09 x 0.69 in)    1
140.8 x 72.4 x 10 mm (5.54 x 2.85 x 0.39 in)      1
7.8 mm thickness                                  1
161.9 x 76.4 x 8.8 mm (6.37 x 3.01 x 0.35 in)     1
143.6 x 66.8 x 7.9 mm (5.65 x 2.63 x 0.31 in)     1
Name: body_dimensions, Length: 633, dtype: int64


In [44]:
# Extract dimensions by splitting the string and removing unnecessary string parts
dimensions = mobiles_info_merged['body_dimensions'].str.split('(',expand=True)[0]
dimensions = dimensions.str.replace(' mm ','')
# Split height, width and depth each in a column
dimensions_splitted = dimensions.str.split('x',expand=True)
# Change col values to numeric
dimensions_splitted.apply(pd.to_numeric, errors='coerce')
# Specify col names
dimensions_splitted.columns = ['body_height(mm)', 'body_width(mm)','body_depth(mm)']
# Add cols to required_cols
if(dimensions_splitted.columns[0] not in required_cols):
    required_cols.extend(dimensions_splitted.columns)

# Remove any special cases after splitting and convert columns to floats
for col in dimensions_splitted.columns:
    # Any non-numeric characters
    dimensions_splitted[col] = dimensions_splitted[col].replace(to_replace ='[^.0-9]', value = '', regex = True)
    # Any invalid value - example 16.360.2
    dimensions_splitted[col] = dimensions_splitted[col].replace(to_replace ='.*\.+.*\.+.*', value = np.nan, regex = True)
    # Remove spaces
    dimensions_splitted[col] = dimensions_splitted[col].str.rstrip()
    dimensions_splitted[col] = dimensions_splitted[col].str.lstrip()
    # Remove None or empty values
    dimensions_splitted[col] = dimensions_splitted[col].replace('',np.nan)
    dimensions_splitted[col].fillna(value=np.nan, inplace=True)
    # Convert col type to float
    dimensions_splitted[col] = dimensions_splitted[col].astype(float)
# Show Result
print(dimensions_splitted.dtypes)
dimensions_splitted.head()

body_height(mm)    float64
body_width(mm)     float64
body_depth(mm)     float64
dtype: object


,body_height(mm),body_width(mm),body_depth(mm)
0,143.5,69.8,8.0
1,124.9,64.0,9.6
2,38.6,33.3,10.5
3,42.0,35.9,10.5
4,38.6,33.3,10.5


In [45]:
# Merge Dimensions df to the original mobiles_info_with_techs df
mobiles_info_with_techs = pd.concat([mobiles_info_with_techs, dimensions_splitted], axis=1)
mobiles_info_with_techs

,brand,mob_name,total_reviews,rating,price,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,...,battery_stand.by,launch_year,GSM,CDMA,HSPA,EVDO,LTE,body_height(mm),body_width(mm),body_depth(mm)
0,Acer,Liquid Jade Z,4,1.500000,129.99,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,"2015, March",...,Up to 410 h (2G) / Up to 400 h (3G),2015.0,1,0,1,0,1,143.5,69.8,8.0
1,Acer,Liquid M2,15,3.200000,34.95,GSM / HSPA,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2015, March",...,Up to 200 h,2015.0,1,0,1,0,0,124.9,64.0,9.6
2,Apple,Watch 38mm,81,4.395062,367.63,No cellular connectivity,NaN,No,No,"2014, September",...,Up to 18 h (mixed usage),2014.0,0,0,0,0,0,38.6,33.3,10.5
3,Apple,Watch 42mm,78,4.423077,775.00,No cellular connectivity,NaN,No,No,"2014, September",...,Up to 18 h (mixed usage),2014.0,0,0,0,0,0,42.0,35.9,10.5
4,Apple,Watch Sport 38mm,535,4.272897,269.99,No cellular connectivity,NaN,No,No,"2014, September",...,Up to 18 h (mixed usage),2014.0,0,0,0,0,0,38.6,33.3,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,alcatel,Pixi 3,85,4.176471,79.95,No cellular connectivity,GSM 850 / 900 / 1800 / 1900,No,No,"2015, September",...,NaN,2015.0,0,0,0,0,0,255.5,155.0,9.5
664,alcatel,Pixi 4,13,4.000000,149.99,GSM / HSPA,GSM 850 / 900 / 1800 / 1900,Yes,Yes,"2016, June",...,Up to 650 h (2G) / Up to 650 h (3G),2016.0,1,0,1,0,0,140.7,72.7,9.5
665,alcatel,Pop 3,17,3.352941,77.10,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2016, February",...,NaN,2016.0,1,0,1,0,1,151.9,77.8,9.5
666,verykool,i121,5,2.600000,10.07,GSM,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,No,2013. Released 2013,...,NaN,NaN,1,0,0,0,0,109.0,44.0,13.0


In [46]:
# Check for body_weight type and values
print(mobiles_info_merged['body_weight'].dtype)
print(mobiles_info_merged['body_weight'].value_counts())

object
120 g (4.23 oz)      14
130 g (4.59 oz)      14
150 g (5.29 oz)      12
155 g (5.47 oz)      12
145 g (5.11 oz)      11
                     ..
122.5 g (4.30 oz)     1
199 g (7.02 oz)       1
142.5 g (5.04 oz)     1
194 g (6.84 oz)       1
84.5 g (2.96 oz)      1
Name: body_weight, Length: 225, dtype: int64


In [47]:
# Get weight no from string
weights = mobiles_info_merged['body_weight'].str.split('g',expand=True)[0]
# Make new col in original df with weights values
mobiles_info_with_techs['body_weight(g)'] = weights
# Remove unnecessary spaces
mobiles_info_with_techs['body_weight(g)'] = mobiles_info_with_techs['body_weight(g)'].str.replace(' ', '')
# Change the col tyoe to numeric
mobiles_info_with_techs['body_weight(g)'] = pd.to_numeric(mobiles_info_with_techs['body_weight(g)'], errors='coerce')
# Add col to required_cols
if('body_weight(g)' not in required_cols):
    required_cols.append('body_weight(g)')
# Show result   
mobiles_info_with_techs['body_weight(g)']

0      110.0
1      119.0
2       40.0
3       50.0
4       25.0
       ...  
663      NaN
664    148.0
665    174.0
666     65.0
667    157.0
Name: body_weight(g), Length: 668, dtype: float64

In [48]:
def create_category_column_from_value(col,value,df,new_col_name):
    
    '''
    INPUT
    col - column needed to be converted as categorial column
    value - The value to search for in col values
    df - the df where the categorial column should be added
    new_col_name - The new categorial column to be added to the df
    
    This function produces from a given column a new categorial column that is indicated by searching for
    a certain value in the col values.
    '''
    # New empty list
    new_col_list = []
    for sim_line in col:
        # Check if value in col value
        if(value in str(sim_line)):
            # Add 1 to list
            new_col_list.append(1)
        else:
            # Add 0 to list
            new_col_list.append(0)
    # Add new list with new col name to df
    df[new_col_name] = new_col_list

    # Add new col to required_cols
    if(new_col_name not in required_cols):
        required_cols.append(new_col_name)
    # Show result
    print(df[new_col_name])

In [49]:
# Check for body_sim type and values
print(mobiles_info_merged['body_sim'].dtype)
print(mobiles_info_merged['body_sim'].value_counts())

object
Mini-SIM                                                                                   199
Micro-SIM                                                                                  101
Nano-SIM                                                                                    75
Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)                                 40
Dual SIM (Micro-SIM, dual stand-by)                                                         37
Dual SIM (Nano-SIM, dual stand-by)                                                          33
Single SIM (Nano-SIM) or Hybrid Dual SIM (Nano-SIM, dual stand-by)                          32
Dual SIM (Mini-SIM)                                                                         18
Dual SIM (Mini-SIM, dual stand-by)                                                          18
Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)                               17
Hybrid Dual SIM (Nano-SIM, dual stand-by)  

In [50]:
# Create a new categorial column indicating dual-sim feature from body_sim col
create_category_column_from_value(mobiles_info_merged['body_sim'],'Dual SIM',mobiles_info_with_techs,'dual_sim')

0      1
1      1
2      0
3      0
4      0
      ..
663    0
664    0
665    1
666    1
667    1
Name: dual_sim, Length: 668, dtype: int64


In [51]:
# Check for body_sim type and values
print(mobiles_info_merged['display_size'].dtype)
print(mobiles_info_merged['display_size'].value_counts())

object
5.0 inches, 68.9 cm (~68.1% screen-to-body ratio)    5
5.5 inches, 83.4 cm (~67.7% screen-to-body ratio)    5
5.2 inches, 74.5 cm (~70.9% screen-to-body ratio)    5
4.7 inches, 60.9 cm (~65.6% screen-to-body ratio)    4
4.3 inches, 51.0 cm (~61.8% screen-to-body ratio)    4
                                                    ..
4.3 inches, 51.0 cm (~54.3% screen-to-body ratio)    1
1.4 inches, 6.2 cm (~13.5% screen-to-body ratio)     1
2.0 inches, 12.4 cm (~23.5% screen-to-body ratio)    1
4.0 inches, 45.5 cm (~58.2% screen-to-body ratio)    1
5.7 inches, 89.6 cm (~67.4% screen-to-body ratio)    1
Name: display_size, Length: 540, dtype: int64


In [52]:
# Remove unneccessary string parts to get the screen-to-body %
display_size = mobiles_info_merged['display_size'].str.split('~',expand=True)[1] 
display_size = display_size.str.split('%',expand=True)[0]
# Change the col values to numeric
display_size = pd.to_numeric(display_size,errors='coerce')
# Create new col with values in the mobiles_info_with_techs df
mobiles_info_with_techs['screen_to_body(%)'] = display_size
# Add col to required cols
if('screen_to_body(%)' not in required_cols):
    required_cols.append('screen_to_body(%)')
# Show values
mobiles_info_with_techs['screen_to_body(%)']

0      68.8
1      57.0
2      55.1
3      56.8
4      55.1
       ... 
663    74.7
664    67.3
665    70.6
666    20.6
667    71.8
Name: screen_to_body(%), Length: 668, dtype: float64

In [53]:
# Check for platform_os type and values
print(mobiles_info_merged['platform_os'].dtype)
print(mobiles_info_merged['platform_os'].value_counts())

object
Android 4.4.2 (KitKat)                                                                  28
Android 5.1 (Lollipop)                                                                  19
Android 4.2 (Jelly Bean)                                                                15
Android 9.0 (Pie)                                                                       15
Android 8.0 (Oreo), upgradable to Android 9.0 (Pie)                                     12
                                                                                        ..
Android 4.3 (Jelly Bean), upgradable to 4.4.2 (KitKat)                                   1
Android 5.1 (Lollipop)Android 6.0 (Marshmallow) - T-Mobile                               1
Android 4.3 (Jelly Bean), upgradable to 4.4.2 (KitKat); Timescape UI                     1
Android 4.1 (Jelly Bean)/ 4.2.2 - C2004/C2005 models, upgradable to 4.3 (Jelly Bean)     1
Symbian, S60 rel. 3.2                                                              

In [54]:
# Create 3 new categorial cols for the 3 mobile operating systems from platform_os col 
# to indicate the platform used in the mobiles
create_category_column_from_value(mobiles_info_merged['platform_os'],'Android',mobiles_info_with_techs,'Android_OS')
create_category_column_from_value(mobiles_info_merged['platform_os'],'iOS',mobiles_info_with_techs,'iOS_OS')
create_category_column_from_value(mobiles_info_merged['platform_os'],'Windows',mobiles_info_with_techs,'Windows_OS')

0      1
1      0
2      0
3      0
4      0
      ..
663    1
664    1
665    1
666    0
667    1
Name: Android_OS, Length: 668, dtype: int64
0      0
1      0
2      0
3      0
4      0
      ..
663    0
664    0
665    0
666    0
667    0
Name: iOS_OS, Length: 668, dtype: int64
0      0
1      1
2      0
3      0
4      0
      ..
663    0
664    0
665    0
666    0
667    0
Name: Windows_OS, Length: 668, dtype: int64


In [55]:
# Check for memory_internal type and values
print(mobiles_info_merged['memory_internal'].dtype)
print(mobiles_info_merged['memory_internal'].value_counts())
#mobiles_info_merged['memory_internal']

object
8 GB, 1 GB RAM                                         53
16 GB, 2 GB RAM                                        40
4 GB, 512 MB RAM                                       32
16 GB, 1 GB RAM                                        29
32 GB, 3 GB RAM                                        25
                                                       ..
8 GB, 1 GB RAM (EU) or 4 GB, 512 MB (North America)     1
8 GB, 128 MB RAM                                        1
32 GB, 128 MB RAM                                       1
150 MB, 256 MB RAM, 256 MB ROM                          1
128 MB, 64 MB RAM                                       1
Name: memory_internal, Length: 273, dtype: int64


In [56]:
def find_matching_str_in_col_strings(col,regex,split_str,new_col_name,df):
    
    '''
    INPUT
    col - column needed to get value from
    regex - regular expression used to extract the part needed for the col value
    split_str - the common string in all col values that needs to be removed to get the numeric value 
    new_col_name - The new column to be added with the new values to the df
    df - the df where the categorial column should be added
    
    This function produces from a given column a new column with numeric values extracted from 
    the original col after cleaning by regex and common string.
    '''
    # Create new list
    new_col = []
    for item in col:
        # Find matching object using regex
        match = re.search(regex,str(item))
        if(match != None):
            # Get matching string value
            match_string = match.group(0)
            # Change the value to numeric after removing the common string 
            match_string_req = int(str(match_string).split(split_str)[0])
            # Add the value to list
            new_col.append(match_string_req)
        else:
            # Add nan if no matches found
            new_col.append(np.nan)
    # Create new col with the chosen new_col_name and the created list as values
    df[new_col_name] = new_col
    # Add new_col_name to required_cols
    if(new_col_name not in required_cols):
        required_cols.append(new_col_name)
    # Show output col
    print(df[new_col_name])

# Get RAM Size from memory_internal col by regular expression checking 
# for no. followed by 'GB RAM' with optional space between them
find_matching_str_in_col_strings(mobiles_info_merged['memory_internal'],r'\d+?\s?GB\sRAM','GB'
                                                             ,'RAM_size(GB)',mobiles_info_with_techs)

0      1.0
1      1.0
2      NaN
3      NaN
4      NaN
      ... 
663    1.0
664    1.0
665    1.0
666    NaN
667    1.0
Name: RAM_size(GB), Length: 668, dtype: float64


In [57]:
# Check for selfie_camera_single type and values
print(mobiles_info_merged['selfie_camera_single'].dtype)
print(mobiles_info_merged['selfie_camera_single'].value_counts())

object
VGA                                                 71
2 MP                                                52
5 MP                                                51
1.3 MP                                              36
5 MP, f/2.2                                         13
                                                    ..
12 MP, 1.25µm                                        1
5 MP (USA)  8 MP (other markets)                     1
5 or 8 MP                                            1
12 MP, f/2.0, 26mm (wide), 1/2.8", 1.25<c2><b5>m     1
24 MP, f/2.0, 27mm (wide), 1/2.8", 0.9µm             1
Name: selfie_camera_single, Length: 129, dtype: int64


In [58]:
# Get Selfie Camera MP value from selfie_camera_single col by regular expression checking 
# for no. followed by 'MP' with optional space between them
find_matching_str_in_col_strings(mobiles_info_merged['selfie_camera_single'],r'\d+?\s?MP','MP'
                                                             ,'selfie_cam(MP)',mobiles_info_with_techs)

0      5.0
1      2.0
2      NaN
3      NaN
4      NaN
      ... 
663    2.0
664    5.0
665    2.0
666    NaN
667    2.0
Name: selfie_cam(MP), Length: 668, dtype: float64


In [59]:
# Get Main Camera MP value from main_camera_single col by regular expression checking 
# for no. followed by 'MP' with optional space between them
find_matching_str_in_col_strings(mobiles_info_merged['main_camera_single'],r'\d+?\s?MP','MP'
                                                             ,'main_cam(MP)',mobiles_info_with_techs)

0      13.0
1       5.0
2       NaN
3       NaN
4       NaN
       ... 
663     5.0
664     8.0
665     5.0
666     NaN
667     8.0
Name: main_cam(MP), Length: 668, dtype: float64


In [60]:
# Check for memory_internal type and values
print(mobiles_info_merged['battery'].dtype)
print(mobiles_info_merged['battery'].value_counts())

object
Non-removable Li-Ion 3000 mAh battery    32
Removable Li-Ion 1500 mAh battery        25
Non-removable Li-Po 4000 mAh battery     18
Removable Li-Ion 1000 mAh battery        17
Removable Li-Ion 2000 mAh battery        14
                                         ..
Removable Li-Ion 2330 mAh battery         1
Non-removable Li-Ion 3230 mAh battery     1
Non-removable Li-Po 2050 mAh battery      1
Non-removable Li-Ion 1305 mAh battery     1
Non-removable Li-Ion 3330 mAh battery     1
Name: battery, Length: 281, dtype: int64


In [61]:
# Get Battery Power Value from battery col by regular expression checking 
# for no. followed by 'mAh' with optional space between them
find_matching_str_in_col_strings(mobiles_info_merged['battery'],r'\d+?\s?mAh','mAh'
                                                             ,'battery(mAh)',mobiles_info_with_techs)

0      2300.0
1      1300.0
2       205.0
3       250.0
4       205.0
        ...  
663    4060.0
664    2000.0
665    2910.0
666     650.0
667    2300.0
Name: battery(mAh), Length: 668, dtype: float64


In [62]:
# Add the reviews cols to the required_cols list
if('total_reviews' not in required_cols):
    required_cols.extend(['total_reviews','rating'])

In [63]:
# Extract only required cols from the mobiles_info_with_techs df after cleaning
mobile_features_with_revs = mobiles_info_with_techs[required_cols]
mobile_features_with_revs

,brand,mob_name,price,launch_year,GSM,CDMA,HSPA,EVDO,LTE,body_height(mm),...,screen_to_body(%),Android_OS,iOS_OS,Windows_OS,RAM_size(GB),selfie_cam(MP),main_cam(MP),battery(mAh),total_reviews,rating
0,Acer,Liquid Jade Z,129.99,2015.0,1,0,1,0,1,143.5,...,68.8,1,0,0,1.0,5.0,13.0,2300.0,4,1.500000
1,Acer,Liquid M2,34.95,2015.0,1,0,1,0,0,124.9,...,57.0,0,0,1,1.0,2.0,5.0,1300.0,15,3.200000
2,Apple,Watch 38mm,367.63,2014.0,0,0,0,0,0,38.6,...,55.1,0,0,0,NaN,NaN,NaN,205.0,81,4.395062
3,Apple,Watch 42mm,775.00,2014.0,0,0,0,0,0,42.0,...,56.8,0,0,0,NaN,NaN,NaN,250.0,78,4.423077
4,Apple,Watch Sport 38mm,269.99,2014.0,0,0,0,0,0,38.6,...,55.1,0,0,0,NaN,NaN,NaN,205.0,535,4.272897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,alcatel,Pixi 3,79.95,2015.0,0,0,0,0,0,255.5,...,74.7,1,0,0,1.0,2.0,5.0,4060.0,85,4.176471
664,alcatel,Pixi 4,149.99,2016.0,1,0,1,0,0,140.7,...,67.3,1,0,0,1.0,5.0,8.0,2000.0,13,4.000000
665,alcatel,Pop 3,77.10,2016.0,1,0,1,0,1,151.9,...,70.6,1,0,0,1.0,2.0,5.0,2910.0,17,3.352941
666,verykool,i121,10.07,NaN,1,0,0,0,0,109.0,...,20.6,0,0,0,NaN,NaN,NaN,650.0,5,2.600000


In [64]:
# A final refinement is to remove smart devices that cannot act as mobile phone since adding them to 
# our model will be a bit misleading to the main idea
print(mobile_features_with_revs.shape)
mobile_features_with_revs_fitered = mobile_features_with_revs.loc[(mobile_features_with_revs['GSM'] == 1) | (mobile_features_with_revs['CDMA'] == 1) | 
                              (mobile_features_with_revs['HSPA'] == 1) | (mobile_features_with_revs['EVDO'] == 1)|
                              (mobile_features_with_revs['LTE'] == 1)]
mobile_features_with_revs_fitered.shape

(668, 24)


(660, 24)

## 6. Save the clean dataset into an sqlite database.

In [65]:
# Create engine
engine = create_engine('sqlite:///Mobile_DB.db')

# Drop all tables that existed before
meta = MetaData(engine)
meta.reflect()
meta.drop_all()

# Add cleaned data to new table
mobile_features_with_revs_fitered.to_sql('Mobiles_Features_Reviews', engine, index=False)

That's it for the ETL Phase, check the next notebook for the data analysis :D